# Notebook 05 — Research Templates

This notebook answers:
- What typically happens before recessions?
- How do metrics behave by regime?
- Is a series leading, lagging, or coincident?

Everything here is designed to be:
- copy-pasteable
- parameter-driven
- reusable in dashboards and Part II

In [1]:
# imports

import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent
sys.path.append(str(PROJECT_ROOT))

from macro_utils.transforms import zscore
from macro_utils.utils import (
    build_transformed_dataset,
    prepare_plot_df,
)
from macro_utils.regimes import build_macro_regimes
from macro_utils.events import event_window

DATA_RAW = PROJECT_ROOT / "data" / "raw"

In [2]:
# load data

monthly = pd.read_csv(
    DATA_RAW / "fred_monthly.csv",
    index_col=0,
    parse_dates=True
)

In [3]:
# transformed dataset

df = build_transformed_dataset(monthly)
df = build_macro_regimes(df)

plot_df = prepare_plot_df(df)

df.head()

c:\Users\JR\OneDrive\Mini PC\FRED\macro_utils\transforms.py:5: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  return series.pct_change(periods) * 100
c:\Users\JR\OneDrive\Mini PC\FRED\macro_utils\transforms.py:8: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  return series.pct_change(1) * 100


,GDP_YoY,CPI_YoY,CPI_MoM,UNRATE,FEDFUNDS,Growth_Regime,Inflation_Regime,Policy_Regime,Macro_Regime
DATE,,,,,,,,,
1991-01-31,2.766649,5.647059,0.372578,6.4,6.91,Expansion,Inflationary,Easing,Expansion / Inflationary / Easing
1991-02-28,2.766649,5.312500,0.074239,6.6,6.25,Expansion,Inflationary,Easing,Expansion / Inflationary / Easing
1991-03-31,2.766649,4.821151,0.000000,6.8,6.12,Expansion,Inflationary,Easing,Expansion / Inflationary / Easing
1991-04-30,2.799215,4.809930,0.222552,6.7,5.91,Expansion,Inflationary,Easing,Expansion / Inflationary / Easing
1991-05-31,2.799215,5.034857,0.370096,6.9,5.78,Expansion,Inflationary,Easing,Expansion / Inflationary / Easing


In [4]:
# regime conditioning

def regime_summary(df, column):
    return (
        df
        .groupby("Macro_Regime")[column]
        .agg(["mean", "std", "count"])
        .sort_values("count", ascending=False)
    )

In [5]:
# GDP growth by regime

regime_summary(df, "GDP_YoY")

,mean,std,count
Macro_Regime,,,
Expansion / Inflationary / Easing,5.042124,2.433090,112
Expansion / Inflationary / Tightening,6.582467,2.066896,108
Expansion / Disinflationary / Tightening,4.458728,1.084269,103
Expansion / Disinflationary / Easing,4.281563,1.625839,78
Contraction / Disinflationary / Easing,-2.722058,2.245557,17
Contraction / Inflationary / Easing,-0.726120,NaN,1


In [6]:
# Metric by regime

fig = px.box(
    plot_df,
    x="Macro_Regime",
    y="GDP_YoY",
    title="GDP YoY by Macro Regime",
    points="outliers"
)

fig.update_layout(height=450)
fig.show()

In [7]:
# Recession Event Study

event_dates = df.index[
    (df["Growth_Regime"] == "Contraction") &
    (df["Growth_Regime"].shift(1) == "Expansion")
]

event_dates[:5]

DatetimeIndex(['2008-10-31', '2020-04-30'], dtype='datetime64[ns]', name='DATE', freq=None)

In [8]:
def plot_event_study(series, event_dates, window=12, title=None):
    windows = event_window(series, event_dates, window)

    if windows.empty or len(windows) < 3:
        print("Insufficient events")
        return

    mean_path = windows.mean()
    p10 = windows.quantile(0.10)
    p90 = windows.quantile(0.90)

    plt.figure(figsize=(10, 4))
    plt.plot(mean_path.index, mean_path, lw=3, label="Mean")
    plt.fill_between(mean_path.index, p10, p90, alpha=0.25, label="10–90% band")
    plt.axvline(0, color="black", lw=1)
    plt.title(title)
    plt.legend()
    plt.show()

In [9]:
# Unemployment around contractions

plot_event_study(
    df["UNRATE"],
    event_dates,
    window=12,
    title="Unemployment Around Growth Contractions"
)

Insufficient events


In [10]:
# z-score extremes

z = zscore(df["CPI_YoY"])

extremes = pd.DataFrame({
    "value": df["CPI_YoY"],
    "z": z,
})

extremes.sort_values("z").head()

,value,z
DATE,,
1998-02-28,1.440200,-3.126078
2008-12-31,-0.022228,-3.042098
1997-12-31,1.697046,-2.990273
1998-03-31,1.376721,-2.980927
1998-01-31,1.631117,-2.923723


In [11]:
# z-scores time series

fig = px.line(
    prepare_plot_df(extremes),
    x="DATE",
    y="z",
    title="CPI YoY Z-Score"
)

fig.add_hline(y=2, line_dash="dash")
fig.add_hline(y=-2, line_dash="dash")

fig.show()

In [12]:
# Regime x Event Intersection

df.loc[event_dates][
    ["GDP_YoY", "CPI_YoY", "UNRATE", "FEDFUNDS", "Macro_Regime"]
].head()

,GDP_YoY,CPI_YoY,UNRATE,FEDFUNDS,Macro_Regime
DATE,,,,,
2008-10-31,-0.726120,3.731058,6.5,0.97,Contraction / Inflationary / Easing
2020-04-30,-6.727971,0.313047,14.8,0.05,Contraction / Disinflationary / Easing
